# Data Collection

This notebook makes collecting data from an AutoAuto car easy.

### Helper functions

- `capture_road_view()`: Capture, clip, resize and grayscale the car's "road view"
- `capture_and_tag(tag)`: Save a snapshot to a category folder of the car's "road view"
- `delete_last()`: Oops! Delete the last captured image if it's bad data

In [4]:
import car
import cv2
import socket
import time
import os

data_directory = "capture_data"
hostname = socket.gethostname()
lastfile = None

tags = [ "forward", "slight_left", "left", "sharp_left", "slight_right", "right", "sharp_right" ]
for directory in tags:
    if not os.path.exists(os.path.join(data_directory, directory)):
        os.makedirs(os.path.join(data_directory, directory))

def capture_road_view():
    return cv2.cvtColor(cv2.resize(car.capture()[120:], (64, 64), interpolation = cv2.INTER_CUBIC), cv2.COLOR_BGR2GRAY)
    
    
def capture_and_tag(tag):
    global lastfile
    filename = os.path.join(data_directory, tag, hostname + str(int(time.time())) + ".png")
    cv2.imwrite(filename, capture_road_view())
    lastfile = filename
    return filename

def delete_last():
    if os.path.exists(lastfile):
        os.remove(lastfile)


### UI Functions

Run this cell to make the user interface

In [2]:
import ipywidgets as widgets
from car import steering, throttle

last_direction = 0.0

# JEAIME DEBUG THIS PLZZZ!!! it's not actually backing up
def undo_capture(button):
    global last_direction
    delete_last()
    steering.set_steering(last_direction)
    throttle.set_throttle(-70)
    time.sleep(0.1)
    throttle.set_throttle(0)
    time.sleep(0.2)
    
def drive(direction):
    global last_direction    
    steering.set_steering(direction)
    throttle.set_throttle(70)
    time.sleep(0.1)
    throttle.set_throttle(0)
    time.sleep(0.2)
    last_direction = direction

def capture_drive(tag, direction):
    capture_and_tag(tag)
    drive(direction)


def capture_forward(button):
    capture_drive("forward", 0)
    
def capture_slight_left(button):
    capture_drive("slight_left", 20)

def capture_left(button):
    capture_drive("left", 35)

def capture_sharp_left(button):
    capture_drive("sharp_left", 50)
    
def capture_slight_right(button):
    capture_drive("slight_right", -20)
    
def capture_right(button):
    capture_drive("right", -35)
    
def capture_sharp_right(button):
    capture_drive("sharp_right", -50)
    
def make_button(description, handler):
    button = widgets.Button(description=description)
    button.on_click(handler)
    return button

commands = [ ("Sharp Left", capture_sharp_left), ("Left", capture_left), ("Slight Left", capture_slight_left),
           ("Forward", capture_forward),
           ("Slight Right", capture_slight_right), ("Right", capture_right), ("Sharp Right", capture_sharp_right) ]
box = widgets.HBox([ make_button(c[0], c[1]) for c in commands ])
undo_button = widgets.Button(description="Undo")
undo_button.on_click(undo_capture)
display(box)
display(undo_button)


HBox(children=(Button(description='Sharp Left', style=ButtonStyle()), Button(description='Left', style=ButtonStyle()), Button(description='Slight Left', style=ButtonStyle()), Button(description='Forward', style=ButtonStyle()), Button(description='Slight Right', style=ButtonStyle()), Button(description='Right', style=ButtonStyle()), Button(description='Sharp Right', style=ButtonStyle())))

Button(description='Undo', style=ButtonStyle())

Instantiated a camera object!
Captured 1 frame.
